In [1]:
from Util import get_openai_api_key
OPEN_API_KEY= get_openai_api_key()

In [2]:
llm_config={
    "config_list":{
        "model":"gpt-3.5-turbo",
        "api_key":OPEN_API_KEY
    }
}

In [16]:
import pandas as pd

from io import StringIO
df= pd.read_csv("english_sentences_1000.csv")

output=df.to_string(index=False)
output

'                              sentence\n                A dog analyzes a song.\n    The scientist eats the experiment.\n            The scientist eats a song.\n           An artist analyzes a house.\n               My friend eats a house.\n        My friend eats the experiment.\n         Our neighbor jumps some food.\n            An artist draws the paper.\n             The teacher runs a house.\n             The robot runs some food.\n                A dog jumps a picture.\n            Our neighbor draws a song.\n               The cat runs some food.\n            The robot draws a picture.\n          The scientist sings a house.\n         Our neighbor draws the paper.\n                  The cat eats a book.\n            My friend draws the paper.\n         The scientist eats a picture.\n            The robot analyzes a book.\n         Our neighbor jumps the paper.\n           The robot builds the paper.\n            The robot draws the paper.\n            The teacher sings a house.\

In [30]:
from fpdf import FPDF
lines = [line.strip() for line in output.strip().split('\n')]
sentences = [line for line in lines if line.lower() != "sentence" and line != ""]

# Step 2: Create PDF and set up font
pdf = FPDF()
pdf.add_page()
pdf.set_font("Arial", size=12)

# Step 3: Add sentences to PDF
for sentence in sentences:
    pdf.multi_cell(0, 10, sentence)

# Step 4: Save
pdf.output("outer_sentences.pdf")

print("✅ PDF created successfully with all sentences.")

# Step 2: Save to a .txt file
#with open("sentences.pdf", "w") as f:
 #   for sentence in sentences:
  #      f.write(sentence + "\n")
#
#print("✅ sentences.txt saved successfully.")

✅ PDF created successfully with all sentences.


In [38]:
from autogen import ConversableAgent

assistant= ConversableAgent(
    name="assistant",
    system_message=""" 
    you are a multilingual table assistant.
Your task is to read a table containing English sentences, translate each sentence into Tamil, and add the translations as a new column.

Instructions:

The input table will have a column named english_sentence (or similar).

Translate each sentence into natural, fluent Tamil.

Add a new column named tamil_translation.

Return the updated table with both columns.

Do not include any explanations, markdown, or formatting — just the raw table.
    """,
    llm_config=llm_config,
    human_input_mode="NEVER",
)

In [39]:
from autogen.retrieve_utils import extract_text_from_pdf

docs_content= extract_text_from_pdf("outer_sentences.pdf")

In [24]:
#pip install fpdf

In [40]:
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent

ragproxyagent=RetrieveUserProxyAgent(
    name="ragproxyagent",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=2,
    code_execution_config=False,
    retrieve_config={
        "task":"qa",
        "docs_path":["outer_sentences.pdf"],
        "docs_content":docs_content,
        "model":"gpt-3.5-turbo",
        "chunks_token_size":200,
        "overwrite":True,
        "get_or_create":True
    }
)

In [41]:
assistant.reset()
question_qa="Can you translate the english sentence to the tamil"
ragproxyagent.initiate_chat(assistant,message=ragproxyagent.message_generator,problem=question_qa)

max_tokens is too small to fit a single line of text. Breaking this line:
	A dog analyzes a song. ...
Failed to split docs with must_break_at_empty_line being True, set to False.
2025-07-02 18:57:09,801 - autogen.agentchat.contrib.retrieve_user_proxy_agent - INFO - Found 4 chunks.
2025-07-02 18:57:09,804 - autogen.agentchat.contrib.vectordb.chromadb - INFO - No content embedding is provided. Will use the VectorDB's embedding function to generate the content embedding.
Number of requested results 20 is greater than number of elements in index 4, updating n_results = 4


Trying to create collection.
VectorDB returns doc_ids:  [['cba477c6', '49705a8c', '24a2640f', '578502ea']]
Adding content of doc cba477c6 to context.
Adding content of doc 49705a8c to context.
ragproxyagent (to assistant):

You're a retrieve augmented chatbot. You answer user's questions based on your own knowledge and the
context provided by the user.
If you can't answer the question with or without the current context, you should reply exactly `UPDATE CONTEXT`.
You must give as short an answer as possible.

User's question is: Can you translate the english sentence to the tamil

Context is: My friend writes a house.
Our neighbor runs a book.
The teacher sings the report.
The cat sings the report.
The scientist builds the report.
The teacher analyzes a book.
The cat jumps the paper.
The robot jumps a book.
The cat sings a song.
An artist builds the paper.
The scientist eats the experiment.
Our neighbor eats the report.
Our neighbor sings a picture.
My friend jumps some food.
The scien

ChatResult(chat_id=None, chat_history=[{'content': "You're a retrieve augmented chatbot. You answer user's questions based on your own knowledge and the\ncontext provided by the user.\nIf you can't answer the question with or without the current context, you should reply exactly `UPDATE CONTEXT`.\nYou must give as short an answer as possible.\n\nUser's question is: Can you translate the english sentence to the tamil\n\nContext is: My friend writes a house.\nOur neighbor runs a book.\nThe teacher sings the report.\nThe cat sings the report.\nThe scientist builds the report.\nThe teacher analyzes a book.\nThe cat jumps the paper.\nThe robot jumps a book.\nThe cat sings a song.\nAn artist builds the paper.\nThe scientist eats the experiment.\nOur neighbor eats the report.\nOur neighbor sings a picture.\nMy friend jumps some food.\nThe scientist runs the paper.\nThe teacher draws the paper.\nThe cat builds the paper.\nThe cat runs a book.\nOur neighbor analyzes some food.Our neighbor draws